# Exercise 2: Embedding models

In this exercise, we'll explore embedding models.

Most embedding models handle text input and output floating point numbers. 

The **dimensions** is the number of dimensions in that model. The "embeddings", or "vectors" are the floating point numbers.

Like the chat completions client, the OpenAI SDK has become the standard for how people use embedding models.

For this exercise, I have pre-computed the embeddings for a "database" of 166 items of clothing for a shop (see data/clothes.json). This will save you time. 

For local development, you can either use `nomic-embed-text` (v1) which is English-only, or `granite-embedding:278m` which is multilingual. Nomic has fewer parameters so it requires less memory to calculate embeddings.

```bash
ollama pull nomic-embed-text
ollama pull granite-embedding:278m
```

In [11]:
# Your first embedding with a model
from openai import OpenAI
import utils

# If you change the environment variables, you need to restart the kernel
api_key = utils.get_api_key()

if utils.MODE == "github":
    model = "text-embedding-3-small"  # A fast, small model
    base_url = "https://models.inference.ai.azure.com"
elif utils.MODE == "ollama":
    # pick which one works for you
    # model = "nomic-embed-text"  # A comparable open-source model
    model = "granite-embedding:278m"  # a multilingual model
    base_url = utils.get_base_url()

# OpenAI client is a class. The old API used to use globals. Sometimes you might see code snippets for the old API. 

client = OpenAI(
    base_url=base_url,
    api_key=api_key,
)

def get_embedding(text, dimensions=1024):
    response = client.embeddings.create(
        input=text,
        model=model,
        dimensions=dimensions, # default is 1536 for text-embedding-3-small. Is not an arbitrary number, is one of the accepted values (256, 512, 1024)
    )
    return response.data[0].embedding

First lets try getting an embedding for a piece of text.

In [12]:
beans_embedding = get_embedding("delicious beans")
print(len(beans_embedding), beans_embedding[:10])

768 [-0.096468195, 0.018140452, -0.040897075, 0.005351554, -0.07713007, -0.012132294, 0.039610785, -0.0083496235, 0.02645198, 0.03154658]


In [13]:
import pandas as pd
from utils.embeddings import cosine_similarity  # See utils/embeddings.py for the cosine similarity function (its not complicated)

data = pd.read_json("data/clothes.json")
if utils.MODE == "ollama":
    if model == "nomic-embed-text":
        data['embedding'] = data.embedding_nomic
    elif model == "granite-embedding:278m":
        data['embedding'] = data.embedding_granite
    else:
        raise ValueError("I didn't precompute those embeddings. ")


def search_df(df, product_description, n=3):
    embedding = get_embedding(product_description, dimensions=1024)
    df['similarities'] = df.embedding.apply(lambda x: cosine_similarity(x, embedding))
    res = df.sort_values('similarities', ascending=False).head(n)
    return res

data

,id,name,description,image,price,embedding,image_embedding,embedding_nomic,embedding_granite
0,1,Azure Dream T-Shirt,"A soft, azure-colored t-shirt made from 100% o...",1.jpeg,19.99,"[-0.018246850000000002, 0.045474958, -0.035167...","[-0.6689453, -0.12243652000000001, -1.8681641,...","[0.019490682000000002, 0.07382982, -0.18183866...","[-0.018246850000000002, 0.045474958, -0.035167..."
1,2,Crimson Night Hoodie,"A warm, crimson hoodie with a kangaroo pocket ...",2.jpeg,39.99,"[-0.041608673000000006, -0.033827413, -0.07790...","[1.4003906, -1.3886718999999998, -2.4453125, -...","[-0.017776126, -0.0026467396, -0.2174231900000...","[-0.041608673000000006, -0.033827413, -0.07790..."
2,3,Golden Glow Dress,"A stunning, golden dress with a flowing silhou...",3.jpeg,59.99,"[-0.041508865000000006, 0.029889135, -0.045875...","[0.98291016, -0.10308838000000001, -0.08532715...","[-0.016198186, 0.055053227, -0.19971026, -0.01...","[-0.041508865000000006, 0.029889135, -0.045875..."
3,4,Emerald Wave Shorts,"Comfortable, emerald green shorts with an elas...",4.jpeg,24.99,"[-0.041499194, -0.004266487, -0.06151339999999...","[1.9863281, 1.3964843999999998, -0.8095703, -2...","[-0.029440032, 0.06919172400000001, -0.1600308...","[-0.041499194, -0.004266487, -0.06151339999999..."
4,5,Sapphire Breeze Jacket,"A lightweight, sapphire blue jacket with a wat...",5.jpeg,49.99,"[-0.020711819000000003, -0.0013402549, -0.0827...","[0.5888671999999999, 0.1887207, -2.6171875, 0....","[0.059686583, -0.019109743000000002, -0.184405...","[-0.020711819000000003, -0.0013402549, -0.0827..."
...,...,...,...,...,...,...,...,...,...
161,162,Amber Glow Cardigan,"A cozy, amber-colored cardigan with chunky kni...",162.jpeg,44.99,"[-0.045380976, -5.7052963e-05, -0.09750344, -0...","[1.59375, 1.4335938, -1.2958984, -1.0791016, -...","[-0.009798418000000001, 0.030412652000000002, ...","[-0.045380976, -5.7052963e-05, -0.09750344, -0..."
162,163,Midnight Eclipse Dress,"A sleek, midnight black dress with a flatterin...",163.jpeg,89.99,"[-0.06840278, -0.004298033000000001, -0.030526...","[1.6962891, 0.7714844, -0.55322266, -1.3916016...","[-0.031131672000000003, 0.020908687000000002, ...","[-0.06840278, -0.004298033000000001, -0.030526..."
163,164,Sunset Haze Scarf,"A lightweight, sunset orange scarf with a soft...",164.jpeg,19.99,"[-0.046309415, 0.0027110893, -0.08797824400000...","[2.1152344, 2.0410156, -1.7548827999999999, 0....","[-0.004934938000000001, -0.0033751142000000003...","[-0.046309415, 0.0027110893, -0.08797824400000..."
164,165,Ivory Dream Blouse,"An elegant, ivory blouse with delicate lace de...",165.jpeg,34.99,"[-0.04824725, 0.031105742000000002, -0.0385377...","[1.9111327999999999, 0.34960938, 0.08654785, -...","[0.00247215, 0.060818754, -0.1681272, -0.01242...","[-0.04824725, 0.031105742000000002, -0.0385377..."


In [16]:
res = search_df(data, 'fishing gear', n=3)

res

,id,name,description,image,price,embedding,image_embedding,embedding_nomic,embedding_granite,similarities
38,39,Coral Reef Swimsuit,"A vibrant, coral pink swimsuit set with adjust...",39.jpeg,34.99,"[-0.045298435000000005, -0.011888897, -0.05256...","[-1.5917968999999998, -0.93603516, -2.6601562,...","[-0.007308654, 0.089311525, -0.15402342, -0.05...","[-0.045298435000000005, -0.011888897, -0.05256...",0.617030
143,144,Cobalt Wave Swim Trunks,"Bold, cobalt blue swim trunks with a drawstrin...",144.jpeg,29.99,"[-0.031639997, -0.027510101000000002, -0.05547...","[0.8334961, 0.8623046999999999, -2.67382809999...","[-0.048130184000000006, 0.058537528000000005, ...","[-0.031639997, -0.027510101000000002, -0.05547...",0.610258
108,109,Coral Reef Swim Trunks,Vibrant coral swim trunks with a quick-dry fab...,109.jpeg,29.99,"[-0.02001357, -0.0224404, -0.047461968, 0.0212...","[0.6298828, 1.2207031, -1.4443359, -0.9589844,...","[-0.01808722, 0.06586623, -0.1732631, -0.07062...","[-0.02001357, -0.0224404, -0.047461968, 0.0212...",0.603685


In [19]:
# This works for many languages, not just English
# Nb: the nomic-embed-text model is barely multilingual. Results will differ greatly with text-embedding-3-small
search_df(data, 'Equipo de pesca', n=3)  # Spanish

,id,name,description,image,price,embedding,image_embedding,embedding_nomic,embedding_granite,similarities
38,39,Coral Reef Swimsuit,"A vibrant, coral pink swimsuit set with adjust...",39.jpeg,34.99,"[-0.045298435000000005, -0.011888897, -0.05256...","[-1.5917968999999998, -0.93603516, -2.6601562,...","[-0.007308654, 0.089311525, -0.15402342, -0.05...","[-0.045298435000000005, -0.011888897, -0.05256...",0.579715
59,60,Coral Reef Swimsuit,"A vibrant, coral pink swimsuit with a flatteri...",60.jpeg,34.99,"[-0.044108644, 0.021162683, -0.050995320000000...","[0.09265137, -0.94628906, -2.1933594, -1.87304...","[0.014526191, 0.06566179500000001, -0.16806918...","[-0.044108644, 0.021162683, -0.050995320000000...",0.550538
143,144,Cobalt Wave Swim Trunks,"Bold, cobalt blue swim trunks with a drawstrin...",144.jpeg,29.99,"[-0.031639997, -0.027510101000000002, -0.05547...","[0.8334961, 0.8623046999999999, -2.67382809999...","[-0.048130184000000006, 0.058537528000000005, ...","[-0.031639997, -0.027510101000000002, -0.05547...",0.544203


In [20]:
search_df(data, '釣りの物' , n=3)  # Japanese

,id,name,description,image,price,embedding,image_embedding,embedding_nomic,embedding_granite,similarities
38,39,Coral Reef Swimsuit,"A vibrant, coral pink swimsuit set with adjust...",39.jpeg,34.99,"[-0.045298435000000005, -0.011888897, -0.05256...","[-1.5917968999999998, -0.93603516, -2.6601562,...","[-0.007308654, 0.089311525, -0.15402342, -0.05...","[-0.045298435000000005, -0.011888897, -0.05256...",0.578352
108,109,Coral Reef Swim Trunks,Vibrant coral swim trunks with a quick-dry fab...,109.jpeg,29.99,"[-0.02001357, -0.0224404, -0.047461968, 0.0212...","[0.6298828, 1.2207031, -1.4443359, -0.9589844,...","[-0.01808722, 0.06586623, -0.1732631, -0.07062...","[-0.02001357, -0.0224404, -0.047461968, 0.0212...",0.575766
113,114,Coral Reef Swim Trunks,Bright coral swim trunks with a tropical print...,114.jpeg,34.99,"[-0.025930135, -0.015090265, -0.045249246, 0.0...","[1.03125, 1.1318359, -3.2910156, -1.4667968999...","[-0.027106052000000002, 0.08375489, -0.1723296...","[-0.025930135, -0.015090265, -0.045249246, 0.0...",0.570445


Even though we searched for the same thing in 3 different languages, the similarity score (right column) was quite different.
The embedding models are multilingual but same-language will score higher.

Computing the similarities for every item is highly intensive, so we can use indexes to cluster vectors together to speed up the search.

# Combining Text Completions and Embeddings to make a RAG bot

We can combine the text-completions (LLM) with the embedding search to find relevant products and include them in the chat.

This information could also be something like a knowledgebase, wiki, or an unstructured data store.

The stages are:

1. Get the request from the user
1. Search the embedding index for similar matches
1. Give those matches to the LLM along with the original question or query
1. Ask it to generate a response
1. Give the response back to the user

In [21]:
if utils.MODE == "github":
    chat_model = "gpt-4.1-nano"  # A fast, small model
elif utils.MODE == "ollama":
    chat_model = "llama3.1"  # llama and ollama are not related. It's a coincidence


def rag_chat(query, n=3):
    # Step 1: Get the embedding for the query
    matches = search_df(data, query, n=n)
    
    # Merge this into a prompt

    system_prompt = f"""
    The user has asked about a product, you are a helpful assistant that can give suggestions about products we have. 

    The matching products are:

    Name: {matches.iloc[0]['name']}
    Description: {matches.iloc[0].description}
    URL: https://www.superpythonshop.com/products/{matches.iloc[0].id}

    Name: {matches.iloc[1]['name']}
    Description: {matches.iloc[1].description}
    URL: https://www.superpythonshop.com/products/{matches.iloc[1].id}

    Name: {matches.iloc[2]['name']}
    Description: {matches.iloc[2].description}
    URL: https://www.superpythonshop.com/products/{matches.iloc[2].id}

    """

    # Step 2: Call the model with the prompt
    response = client.chat.completions.create(
        model=chat_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
        ],
        temperature=0.5,
        n=1,
    )

    # Step 3: Return the response
    return response.choices[0].message.content

from IPython.display import display, Markdown

display(Markdown(rag_chat("I need a warm hat for winter")))


Based on your requirement, I would recommend the "Golden Harvest Beanie" (https://www.superpythonshop.com/products/21). It's made from soft wool and has a warm golden-yellow color that will keep you cozy during the colder months.

If you'd like to consider other options, the "Slate Gray Beanie" (https://www.superpythonshop.com/products/113) is also a great choice. It's another beanie option that's perfect for winter, with a soft and knitted wool material.

Let me know if either of these options appeal to you, or if you'd like more suggestions!

# Task

Your next job is to iterate on this prompt to refine it and improve the suggestions. Try different queries and searches to see what it does.

Try:
- Looking for something silly
- Looking for something that doesn't exist
- Starting an argument with it
- Asking a question with errors
- Asking a question in a different language


Can you get a good prompt?